## 반자동 문서 스캐너 업로드용

### 마우스 이벤트 등록

In [ ]:
import cv2

def mouse_handler(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN: #마우스 왼쪾 버튼 눌렀다면?
        print('왼쪽 버튼 Down')
        print(x,y) #눌려진 좌표를 알 수 있다.
    elif event == cv2.EVENT_LBUTTONUP: # 마우스 왼쪽 버튼 up -> 버튼에서 손가락을 뗐을 때
        print('왼쪽 버튼 Up')
        print(x,y) #눌려진 좌표를 알 수 있다.
    elif event == cv2.EVENT_LBUTTONDBLCLK: #마우스 왼쪽 버튼 더블 클릭
         print('왼쪽 버튼 Double Click') #더블클릭은 DOWN -> UP -> DOWN(Double Click 발생) -> UP 
    elif event == cv2.EVENT_RBUTTONDOWN: #오른쪽 버튼 down
         print('오른쪽 버튼 Down')
            
#    elif event == cv2.EVENT_MOUSEMOVE: #마우스 이동 -> 계속 마우스 이동만 뜨니까 잠시 주석처리 ... 
#         print('마우스 이동')        
        
img = cv2.imread('poker.jpg')
cv2.namedWindow('img') #img란 이름의 윈도우를 먼저 만들어두는 것. 여기에 마우스 이벤트를 처리하기 위한 핸들러 적용
cv2.setMouseCallback('img', mouse_handler )#(img라는 이름을 가진 윈도우에 어떤 이벤트 처리를 하기 위한 핸들러를 달아둠 ->
#이 mouse_handler는 위에서 정의해줘야 한다 원래 있는 함수가 아님

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 최종본

In [3]:
import cv2
import numpy as np


point_list = []#마우스로 입력받은 4개의 지점을 저장하기 위한 list
src_img = cv2.imread('poker.jpg')

COLOR = (0,255,0) #BGR

#선을 그어주기 위해 추가한 부분------------------------
THICKNESS = 3; #선의 두께
drawing = False
#선을 그릴지의 여부 -> 이게 무슨 말일까?
#이 프로그램을 실행하자마자 선을 그리면 안된다. 기본값은 False였다가 지점을 하나 찍고나면 Ture로 바뀌고 그때부터 선을 그리도록한다.

def mouse_handler(event, x, y, flags, param):
    global drawing #전역변수 선언
    dst_img=src_img.copy() #이미지를 카피 why?-> 이 문장이 없이 실행했더니 마우스를 움직일 때 계속 업데이트됨
                        #원본이미지의 복사본 위에 그리기 때문에
    
    if event == cv2.EVENT_LBUTTONDOWN: #마우스 왼쪽 버튼 눌렀다면
        drawing = True #선을 그리기 시작하면?
        point_list.append((x,y))#list에 누른 좌표가 들어감.
    
    if drawing: # !!여기서 만난 문제점 **1
        previous_point = None #시작점(이전에 존재하는 pointer)
        for point in point_list:#클린된 지점을 이미지에 동그라미로 표시하기 위한 for문
            cv2.circle(dst_img, point, 15, COLOR, cv2.FILLED )
            if previous_point:
                cv2.line(dst_img, previous_point, point, COLOR, THICKNESS , cv2.LINE_AA) #line함수를 통해서 그림을 그려줌
            previous_point=point #2번째 점을 3번째 점에 넣고, 3번째 점을 4번째 점에 넣고...
#마지막 2개의 점에 대해서는 직선이 없으므로  

        next_point=(x,y)#마지막 지점으로부터 현재 지점까지 이어주는, 현재 마우스 좌표를 따라오고 있음
        if len(point_list) == 4: #만약 점 4개를 다 찍었다면?
            show_result() #결과를 출력해주는 함수(사용자 정의 함수)
            next_point = point_list[0] #만약 점 4개를 다 연결했다면? 첫 번째 점의 좌표와 연결하기
        cv2.line(dst_img, previous_point, next_point, COLOR, THICKNESS, cv2.LINE_AA)
    
    cv2.imshow('img', dst_img)
    
    
#결과를 출력해주는 사용자 정의 함수인 show_result() 정의하기
def show_result():
    width, height = 530, 710
    src = np.float32(point_list) #ponit_list 안에 있는 4개의 지점을 가지고 float32형태의 array를 만들어 주는 거임 
    dst = np.array([[0,0], [width, 0], [width, height], [0,height]], dtype=np.float32) #output  
    
    matrix = cv2.getPerspectiveTransform(src,dst) #Matrix 얻어옴
    result = cv2.warpPerspective(src_img, matrix, (width,height)) 
    cv2.imshow('result', result) #결과가 출력되는 창
    
cv2.namedWindow('img')
cv2.setMouseCallback('img', mouse_handler )
cv2.imshow('img', src_img)
cv2.waitKey(0)
cv2.destroyAllWindows()